## South china morning post-Web scraping

In [7]:
! pip install beautifulsoup4

  Could not find a version that satisfies the requirement beautifulsoup3 (from versions: )
No matching distribution found for beautifulsoup3
You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Import libraries

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import numpy as np

Load the URL and store it

In [2]:
#Open URL and read it
url = urlopen("http://www.scmp.com/content/search/canada?f[0]=im_field_label_tag%3A544&f[1]=ds_created%3A%5B2008-03-01T00%3A00%3A00Z%20TO%202017-02-29T23%3A59%3A59Z%5D")
content = url.read()

In [3]:
soup = BeautifulSoup(content)

C:\Users\Jose Manuel\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Jose Manuel\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [4]:
# The are 12 results webpages with 10 news articles per webpage.
links=[] #create a list of links to the 5 pages
wbp="http://www.scmp.com"
for divTag in soup.find_all( "div", {"class" : "item-list"}):
    for ulTag in divTag.find_all( "li", {"class" : "pager-item"}):
        links.append(wbp+ulTag.a["href"])

Extract data from each hyperlink and store it in the dictionary

In [12]:
#Initialize empty dictionary to fill with data 
my_dict={}
wbp="http://www.scmp.com"
count_1=0
for link in links: 
    count_1=count_1+1
    #Open link
    url = urlopen(link) #Open webpage of results
    content = url.read()
    soup = BeautifulSoup(content)
    count_2=0
    for divTag in soup.findAll("div", { "class" : "search-result-wrapper"}): #Get the links of the pages of news
        count_2=count_2+1
        page=wbp+divTag.h3.a['href'] 
        try:
            print('Link:',count_1,'Page:',count_2)
            print(page)
            url = urlopen(page)  #Open page of the news article
            content = url.read()
            soup = BeautifulSoup(content)
            content_temp=''
            my_dict['Page-'+str((count_1-1)*10+count_2)]={}
            for divTag_2 in soup.findAll("h1", { "class" : "title"}):   #Get headline
                my_dict['Page-'+str((count_1-1)*10+count_2)]['headline']=divTag_2.text
            for divTag_3 in soup.findAll("div", { "class" : "pane-content"}): #Get content
                for divTag_4 in divTag_3.findAll("p"):
                    try:
                        content_temp=content_temp+' '+divTag_4.text
                    except:
                        continue
            my_dict['Page-'+str((count_1-1)*10+count_2)]['content']= content_temp
        except:
            continue

C:\Users\Jose Manuel\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Jose Manuel\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Link: 1 Page: 1
http://www.scmp.com/property/international/article/2066734/bc-government-plans-lift-15-cent-foreign-buyer-tax-those-work
Link: 1 Page: 2
http://www.scmp.com/news/world/united-states-canada/article/2065300/canadian-property-owner-sues-over-denial-moby-dick
Link: 1 Page: 3
http://www.scmp.com/news/world/united-states-canada/article/2063520/vancouver-airport-reveals-us42-billion-expansion
Link: 1 Page: 4
http://www.scmp.com/property/international/article/2061199/bc-housing-starts-hit-highest-level-decades-last-year
Link: 1 Page: 5
http://www.scmp.com/tech/article/2056856/canada-declares-broadband-internet-access-basic-service
Link: 1 Page: 6
http://www.scmp.com/property/international/article/2062776/bc-has-record-year-home-sales-2016
Link: 1 Page: 7
http://www.scmp.com/news/world/united-states-canada/article/2059523/canadian-winter-weather-pushes-bc-power-consumption
Link: 1 Page: 8
http://www.scmp.com/tech/start-ups/article/2058930/kick-starting-drive-hydrogen-fuel-cell-2

In [20]:
#encode weird characters to ascii
my_dict_1={}
import unicodedata
for x in range(1,121):
    my_dict_1['Page-'+str(x)]={}
    my_dict_1['Page-'+str(x)]['content']=unicodedata.normalize('NFKD',my_dict['Page-'+str(x)]['content']).encode('ascii','ignore')

In [21]:
#encode content to utf-8
my_dict_fin={}
for x in range(1,121):
    my_dict_fin['Page-'+str(x)]={}
    my_dict_fin['Page-'+str(x)]['content'] = str(my_dict_1['Page-'+str(x)]['content'], "utf-8")

In [24]:
#Write CSV file
with open("scmp_news.csv","w") as toWrite:
    writer=csv.writer(toWrite,delimiter=",")
    writer.writerow([ "Headline","Content"])
    for x in range(1,121):
        x=str(x)
        writer.writerow([my_dict['Page-'+x]['headline'], my_dict_fin['Page-'+x]['content']])

Import CSV File and open

In [16]:
#Import CSV file to open
import pandas
news_articles = pandas.read_csv('scmp_news.csv', encoding = "ISO-8859-1")
news_articles

,Headline,Content
0,B.C. government plans to lift 15 per cent fore...,Business in Vancouver A timeline for the c...
1,Canadian property owner sues over denial of Mo...,Business in Vancouver Owners councils ref...
2,Vancouver airport reveals US$4.2 billion expan...,Business in Vancouver Expansions to five t...
3,BC housing starts hit highest level in decades...,Business in Vancouver Provinces yearly to...
4,Canada declares broadband Internet access a ba...,Business in Vancouver The regulator has se...
5,BC has record year for home sales in 2016,Business in Vancouver Canadian provinces ...
6,Canadian winter weather pushes BC power consum...,Business in Vancouver Monday saw the most ...
7,Kick-starting the drive for hydrogen fuel cel...,Business in Vancouver Vancouver company Lo...
8,Vancouver home affordability hits record low b...,Business in Vancouver The city is stillby...
9,British Columbia home to 20 of Canadas top 10...,Business in Vancouver Womens Executive Ne...
